In [ ]:
import node2vecs
import lfr
import cProfile
import networkx as nx
import numpy as np
import pandas as pd
import umap.umap_ as umap
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import importlib

In [52]:
def load_embeddings(pkl_path):
    """Load your three embedding matrices from the pickle."""
    with open(pkl_path, 'rb') as f:
        d = pickle.load(f)
    # Expect keys "dot", "euclidean", "cosine"
    return d['dot'], d['euclidean'], d['cosine']

def load_communities(csv_path):
    """Load your node→community mapping (must be a DataFrame with 'node','community')."""
    df = pd.read_csv(csv_path)
    # Assumes df.node is 0..N-1
    return df['community_id'].values

import pickle
import pandas as pd
mu = 0.3

base = "/l/research/gogandhi.NOBACKUP/alt_means_sans_k/data/experiment_n2v_metric_change_10000_5_3.0_minc50_immutable/"
#file_path = f"Run_1/embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_50_mincomm_50.pkl"
comm_path = f"Run_1/community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_5_mincomm_50.csv"
net_path=f"Run_1/net_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_5_mincomm_50.npz"


# 1) Load

net = sparse.load_npz(base+net_path)

comm = load_communities(base+comm_path)

In [53]:
#2) Generate the embeddings
# profiler = cProfile.Profile()
# profiler.enable()
#-----------------------

num_walks = 10
walk_length = 80
window_size = 10
embedding_dim = 64



similarity_measures = ["euclidean","dot","cosine"]
embeddings = {}
embedding_dim = 64

for sim in similarity_measures:
    print(f"Training model using {sim} similarity...")
    model = node2vecs.TorchNode2Vec(vector_size=embedding_dim, similarity_metric=sim, device = 'cuda:1',num_workers=1)
    model.fit(net)
    embeddings[sim] = model.transform()
    
# Now embeddings["dot"], embeddings["cosine"], and embeddings["euclidean"]
# contain the respective node embeddings.
 
#-----------------------
# profiler.disable()

Training model using euclidean similarity...


100%|██████████████████████████████████████████████████████████████████████████| 31250/31250 [05:23<00:00, 96.71it/s, loss=1.36]


Training model using dot similarity...


100%|█████████████████████████████████████████████████████████████████████████| 31250/31250 [05:10<00:00, 100.62it/s, loss=1.27]


Training model using cosine similarity...


100%|██████████████████████████████████████████████████████████████████████████| 31250/31250 [06:24<00:00, 81.18it/s, loss=1.27]


In [49]:
from sklearn.metrics import roc_auc_score

def sample_pairs(comm, n_pos=50_000, n_neg=50_000, seed=0):
    """Randomly sample n_pos same-community and n_neg different-community pairs."""
    rng = np.random.default_rng(seed)
    N = len(comm)
    same, diff = [], []
    # keep drawing until we have enough
    while len(same) < n_pos or len(diff) < n_neg:
        i, j = rng.integers(0, N, 2)
        if i == j:
            continue
        if comm[i] == comm[j]:
            if len(same) < n_pos:
                same.append((i, j))
        else:
            if len(diff) < n_neg:
                diff.append((i, j))
    pairs = np.array(same + diff, dtype=int)
    labels = np.array([1]*len(same) + [0]*len(diff))
    return pairs, labels

def compute_scores(embeddings, pairs):
    """
    embeddings: dict with keys 'dot', 'euclid', 'cosine' mapping to (N×d) arrays
    pairs: (M×2) array of node‐index pairs
    Returns:
      scores[emb_name][metric] = length‐M score vector
    """
    i_idx, j_idx = pairs[:,0], pairs[:,1]
    scores = {}
    for emb_name, E in embeddings.items():
        # 1) raw dot‐product
        dots = np.sum(E[i_idx] * E[j_idx], axis=1)
        # 2) norms
        norms = np.linalg.norm(E, axis=1)
        # 3) cosine similarity
        cosines = dots / (norms[i_idx] * norms[j_idx] + 1e-12)
        # 4) norm‐only score
        norm_diff = -np.abs(norms[i_idx] - norms[j_idx])
        # 5) Euclid on normalized vectors
        E_unit = E / (norms[:, None] + 1e-12)
        euc_on_norm = -np.linalg.norm(E_unit[i_idx] - E_unit[j_idx], axis=1)

        scores[emb_name] = {
            'dot':        dots,
            'cosine':     cosines,
            'norm_diff':  norm_diff,
            'euc_on_norm': euc_on_norm
        }
    return scores

# --- usage ----

# embeddings is your dict:
#   embeddings = {
#     'dot':   E_dot,
#     'euclid':E_euc,
#     'cosine':E_cos
#   }
# comm is your community array

pairs, labels = sample_pairs(comm)
scores = compute_scores(embeddings, pairs)

# Print out every AUC
for emb_name, metric_dict in scores.items():
    for metric, sc in metric_dict.items():
        auc = roc_auc_score(labels, sc)
        print(f"AUC (trained on {emb_name}, eval {metric}): {auc:.4f}")


AUC (trained on euclidean, eval dot): 1.0000
AUC (trained on euclidean, eval cosine): 1.0000
AUC (trained on euclidean, eval norm_diff): 0.5008
AUC (trained on euclidean, eval euc_on_norm): 1.0000
AUC (trained on dot, eval dot): 0.9862
AUC (trained on dot, eval cosine): 0.9862
AUC (trained on dot, eval norm_diff): 0.4984
AUC (trained on dot, eval euc_on_norm): 0.9862
AUC (trained on cosine, eval dot): 0.9846
AUC (trained on cosine, eval cosine): 0.9846
AUC (trained on cosine, eval norm_diff): 0.4951
AUC (trained on cosine, eval euc_on_norm): 0.9846


In [31]:
import copy
emb_example = copy.deepcopy(embeddings)

In [41]:
emb_path = "/l/research/gogandhi.NOBACKUP/alt_means_sans_k/data/experiment_n2v_metric_cosine_change_10000_50_3.0_minc50/Run_2/embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_0.3_k_50_mincomm_50.pkl"

mu=0.3
base = "/l/research/gogandhi.NOBACKUP/alt_means_sans_k/data/experiment_n2v_metric_change_10000_50_3.0_minc50_immutable/"
#emb_path = f"Run_1/embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_50_mincomm_50.pkl"
comm_path = f"Run_1/community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_50_mincomm_50.csv"
net_path=f"Run_1/net_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_50_mincomm_50.npz"


with open(emb_path, 'rb') as f:
    emb = pickle.load(f)

comm = load_communities(base+comm_path)

In [42]:
# 2) Sample
pairs, labels = sample_pairs(comm)

In [57]:
import numpy as np
def compute_scores(embeddings, pairs):
    """
    embeddings: dict with keys 'dot', 'euclid', 'cosine' mapping to (N×d) arrays
    pairs: (M×2) array of node‐index pairs
    Returns:
      scores[emb_name][metric] = length‐M score vector
    """
    i_idx, j_idx = pairs[:,0], pairs[:,1]
    scores = {}
    for emb_name, E in embeddings.items():
        # 1) raw dot‐product
        dots = np.sum(E[i_idx] * E[j_idx], axis=1)
        # 2) norms
        norms = np.linalg.norm(E, axis=1)
        # 3) cosine similarity
        cosines = dots / (norms[i_idx] * norms[j_idx] + 1e-12)
        # 4) norm‐only score
        norm_diff = -np.abs(norms[i_idx] - norms[j_idx])
        # 5) Euclid on normalized vectors
        E_unit = E / (norms[:, None] + 1e-12)
        euc_on_norm = -np.linalg.norm(E_unit[i_idx] - E_unit[j_idx], axis=1)

        scores[emb_name] = {
            'dot':        dots,
            'cosine':     cosines,
            'norm_diff':  norm_diff,
            'euc_on_norm': euc_on_norm
        }
    return scores

pairs, labels = sample_pairs(comm)
scores = compute_scores(embeddings, pairs)

# Print out every AUC
for emb_name, metric_dict in scores.items():
    for metric, sc in metric_dict.items():
        auc = roc_auc_score(labels, sc)
        print(f"AUC (trained on {emb_name}, eval {metric}): {auc:.4f}")


AUC (trained on euclidean, eval dot): 0.8631
AUC (trained on euclidean, eval cosine): 0.8631
AUC (trained on euclidean, eval norm_diff): 0.4979
AUC (trained on euclidean, eval euc_on_norm): 0.8631
AUC (trained on dot, eval dot): 0.8314
AUC (trained on dot, eval cosine): 0.8314
AUC (trained on dot, eval norm_diff): 0.4981
AUC (trained on dot, eval euc_on_norm): 0.8314
AUC (trained on cosine, eval dot): 0.8300
AUC (trained on cosine, eval cosine): 0.8300
AUC (trained on cosine, eval norm_diff): 0.5025
AUC (trained on cosine, eval euc_on_norm): 0.8300


In [58]:
embeddings

{'euclidean': array([[ 2.94235826e-01,  5.42403385e-02,  2.11512461e-01, ...,
          1.39160514e-01, -7.29556829e-02,  1.50696198e-02],
        [ 2.29102686e-01,  5.97083606e-02,  9.57444981e-02, ...,
          1.36811569e-01, -4.27412167e-02, -1.34613626e-02],
        [ 2.76519626e-01,  5.38738556e-02,  2.06299484e-01, ...,
          1.33773461e-01, -2.24900041e-02,  1.59789270e-04],
        ...,
        [ 2.44028196e-01,  8.15561935e-02,  1.71288416e-01, ...,
          1.17202535e-01, -1.80401222e-03, -3.86278071e-02],
        [ 2.48516396e-01,  8.18043500e-02,  2.11611465e-01, ...,
          1.40022650e-01, -4.44436111e-02,  1.26816211e-02],
        [ 2.51754850e-01,  4.86937575e-02,  2.01626107e-01, ...,
          1.37111336e-01, -3.38552929e-02, -3.97842657e-03]], dtype=float32),
 'dot': array([[-0.06865021,  0.00576509,  0.17189175, ...,  0.13836883,
          0.13703142, -0.02401272],
        [-0.21217427,  0.0264034 , -0.04126452, ..., -0.1270578 ,
          0.07417835, -0.3

In [59]:
1

1

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
